### Considered model: four-bar linkage (= two link manipulator + one link manipulator + rigid coupling)


This Notebook depends on symbtools >=0.2.5

→ `pip install --upgrade symbtools`





In [1]:
%load_ext ipydex.displaytools
%matplotlib inline

import sys
# to load assimulo under wsl2
sys.path.append('/home/kwrede/miniconda3/lib/python3.8/site-packages')
sys.path.append('/home/wredi/miniconda3/lib/python3.8/site-packages')

import os
import sympy as sp
import numpy as npy  # we need `np` later 
from sympy import sin, cos, pi
from sympy.interactive import printing

# using wredsen's symbtools fork (https://github.com/wredsen/symbtools @ DAE_statefeedback), assuming repos SA-Wrede and symbtools share the same parent directory
sys.path.append('../../symbtools/')
import symbtools as st
import symbtools.modeltools as mt
from symbtools.modeltools import Rz # Rotationsmatrix in the xy plane (c, -s; s, c)

import scipy.integrate as sc_integrate
from scipy.optimize import fmin
import matplotlib.pyplot as pl
from assimulo.solvers import ODASSL as SOLVER # Imports the solver ODASSL from Assimulo
from assimulo.problem import Overdetermined_Problem # Imports the problem formulation from Assimulo
import ipydex
plt = pl

printing.init_printing(1)

In [2]:
import pickle

with open("DAE_fully_actuated_crane.pcl", "rb") as pfile:
    mod = pickle.load(pfile)

### Creation of DAE System

In [3]:
parameter_values = list(dict(s2=0.15, m1=0.45, m2=0.557, m3=0.45, J2=0.000221, mc1b=39, mc2b=36.3, d1=9.76, d4=11.2, l0=0.5, l1=0.4, l2=0.3, l3=0.4, g=9.81).items()) ##:
# tbd: J2, d1, d4

parameter_values := [('s2', 0.15),
 ('m1', 0.45),
 ('m2', 0.557),
 ('m3', 0.45),
 ('J2', 0.000221),
 ('mc1b', 39),
 ('mc2b', 36.3),
 ('d1', 9.76),
 ('d4', 11.2),
 ('l0', 0.5),
 ('l1', 0.4),
 ('l2', 0.3),
 ('l3', 0.4),
 ('g', 9.81)]

---

#### DAE system for simulation

In [4]:
# generate numerical DAE system from implicit system equations
dae = mod.calc_dae_eq(parameter_values)
dae.generate_eqns_funcs()

In [5]:
# define closed loop (usage of yy) control function for input
# di*yy[] terms represent dissipation in carts
def new_input_func(yy, t):
    u1 = 39*yy[1] - 9.76*yy[8]
    u2 = 36.3*yy[1] - 11.2*yy[9]
    u3 = 0
    u4 = 0
    return [u1, u2, u3, u4]

# set as input of DAE system
dae.input_func = new_input_func ##:

(dae.input_func) := <function __main__.new_input_func(yy, t)>

---

In [6]:
### check error of constraints for initial values
init_theta = [5.1, 1.059819394358637, -1.5973816338597095, 0.0, 0.0, 0.1, 0.1]

# calculate consistent initial values from latest visualization
yy0, yyd0 = dae.calc_consistent_init_vals(p1=init_theta[0], q1=init_theta[3], q2=init_theta[4], q3=init_theta[5], q4=init_theta[6], _ftol=1e-9) ##:

t0 = 0

# evaluation of implicit DAE System (should be almost 0 if initial values fulfill the model)
dae.model_func(t0, yy0, yyd0) ##:

# check if all values are almost zero (-> initial values fulfill the model)
assert npy.allclose(dae.model_func(t0, yy0, yyd0), 0)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 104
         Function evaluations: 196
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 35
         Function evaluations: 71


LinAlgError: 1-dimensional array given. Array must be at least two-dimensional